In [1]:
using CovidSim_ilm

┌ Info: Precompiling CovidSim_ilm [top-level]
└ @ Base loading.jl:1278


In [120]:
using BenchmarkTools
using Distributions

This was a good start on a new approach to transition. A few details had to be changed, combined
but it did work first time algorithmically.

# Grab a population matrix after 80 days

In [274]:
seed_1_6 = seed_case_gen(1, [0,3,3,0,0], 1, nil, agegrps)

(::CovidSim_ilm.var"#scase#64"{Int64,Array{Int64,1},Int64,Int64,UnitRange{Int64}}) (generic function with 1 method)

In [275]:
alldict, env, series = run_a_sim(80, 38015, showr0=false, silent=true, spreadcases=[], runcases=[seed_1_6]);

*** seed day 1 locale 38015....
(sptime, trtime) = (0.24795550200000002, 1.3013469130000002)


In [276]:
locdat = alldict["dat"]["openmx"][38015];

In [277]:
locdat[1:20,1:4]

20×4 Array{Int64,2}:
 1  1  0  0
 2  1  5  2
 1  1  0  0
 1  1  0  0
 1  1  0  0
 1  1  0  0
 1  1  0  0
 2  1  6  8
 1  1  0  0
 1  1  0  0
 1  1  0  0
 1  1  0  0
 1  1  0  0
 1  1  0  0
 1  1  0  0
 1  1  0  0
 1  1  0  0
 1  1  0  0
 1  1  0  0
 1  1  0  0

In [278]:
dt = alldict["dt"]

Dict{Any,Any} with 5 entries:
  4 => Dict{Any,Any}([9, 5]=>Dict{String,Array{T,1} where T}("probs"=>[0.62, 0.…
  2 => Dict{Any,Any}([9, 5]=>Dict{String,Array{T,1} where T}("probs"=>[0.85, 0.…
  3 => Dict{Any,Any}([9, 5]=>Dict{String,Array{T,1} where T}("probs"=>[0.9, 0.1…
  5 => Dict{Any,Any}([9, 5]=>Dict{String,Array{T,1} where T}("probs"=>[0.5, 0.5…
  1 => Dict{Any,Any}([9, 5]=>Dict{String,Array{T,1} where T}("probs"=>[0.9, 0.1…

In [279]:
dt[1][[9,5]]

Dict{String,Array{T,1} where T} with 3 entries:
  "probs"    => [0.9, 0.1]
  "branches" => Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"=>0.9), Dict(…
  "outcomes" => [3, 7]

In [280]:
dts = Dict(i=>sort(alldict["dt"][i], rev=true) for i in 1:5) 

Dict{Int64,OrderedCollections.OrderedDict{Any,Any}} with 5 entries:
  4 => OrderedCollections.OrderedDict{Any,Any}([25, 8]=>Dict{String,Array{T,1} …
  2 => OrderedCollections.OrderedDict{Any,Any}([25, 8]=>Dict{String,Array{T,1} …
  3 => OrderedCollections.OrderedDict{Any,Any}([25, 8]=>Dict{String,Array{T,1} …
  5 => OrderedCollections.OrderedDict{Any,Any}([25, 8]=>Dict{String,Array{T,1} …
  1 => OrderedCollections.OrderedDict{Any,Any}([25, 8]=>Dict{String,Array{T,1} …

In [281]:
collect(keys(dts[4]))

10-element Array{Any,1}:
 [25, 8]
 [25, 7]
 [19, 8]
 [14, 8]
 [14, 7]
 [14, 6]
 [9, 7]
 [9, 6]
 [9, 5]
 [5, 5]

In [282]:
lags_by_age = Dict{Int,Array{Int,1}}()  # empty
fromconds_by_age = Dict{Int,Array{Int,1}}()  # empty
for i in 1:5
    lags_by_age[i] = [k[1] for k in collect(keys(dts[i]))]
    fromconds_by_age[i] = [k[2] for k in collect(keys(dts[i]))]
end
display(lags_by_age)
display(fromconds_by_age)

Dict{Int64,Array{Int64,1}} with 5 entries:
  4 => [25, 25, 19, 14, 14, 14, 9, 9, 9, 5]
  2 => [25, 25, 19, 14, 14, 14, 9, 9, 9, 5]
  3 => [25, 25, 19, 14, 14, 14, 9, 9, 9, 5]
  5 => [25, 25, 19, 14, 14, 14, 9, 9, 9, 5]
  1 => [25, 25, 19, 14, 14, 14, 9, 9, 9, 5]

Dict{Int64,Array{Int64,1}} with 5 entries:
  4 => [8, 7, 8, 8, 7, 6, 7, 6, 5, 5]
  2 => [8, 7, 8, 8, 7, 6, 7, 6, 5, 5]
  3 => [8, 7, 8, 8, 7, 6, 7, 6, 5, 5]
  5 => [8, 7, 8, 8, 7, 6, 7, 6, 5, 5]
  1 => [8, 7, 8, 8, 7, 6, 7, 6, 5, 5]

In [283]:
infected_idx = findall(locdat[:,1] .== 2);
sorted_by_lag = sortperm(locdat[infected_idx,4], rev=true);

In [284]:
@elapsed for p in infected_idx[sorted_by_lag]  # p is the index to the person
    (pstat, page, plag, pcond) = locdat[p,[cpop_status, cpop_agegrp, cpop_lag, cpop_cond]]

    lagfound = findall(x->x==plag, lags_by_age[page])
    if isempty(lagfound)  # person's lag doesn't match any decision point lag
        # test against laglim, then increment
        if plag == laglim
            @error "person made it to end of laglim and was not removed"
        else
            locdat[p,cpop_lag] += 1
        end
    else
        condfound = findall(x->x==pcond, fromconds_by_age[page][lagfound])
        if isempty(condfound)
            if plag == laglim
                @error "person made it to end of laglim and was not removed"
            else
                locdat[p,cpop_lag] += 1
            end
        else
            # do the transition for lag and from cond and the probabilities of all outcomes at this branch
            dtkey = [plag, pcond]
            probs = dts[page][dtkey]["probs"]
            outcomes = dts[page][dtkey]["outcomes"]
            choice = rand(Categorical(probs), 1)
            tocond = outcomes[choice][]
            if tocond in [dead, recovered]  # change status, leave cond and lag as last state before death or recovery
                locdat[p, cpop_status] = tocond
            else   # change disease condition
                locdat[p, cpop_cond] = tocond
                locdat[p, cpop_lag] += 1  
            end                      
        end
    end    
end

0.177589041

In [285]:
locdat[95515,1:4]

4-element Array{Int64,1}:
 1
 5
 0
 0